# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Functions to interact with OpenAI's API it will take one user message and will give you response.
#  Function to get completion from a user message
#  @param prompt: The user message to get a response for
#  @param model: The model to use for generating the response
#  @return: The content of the response message
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]  # create a list of messages
    # the model expects a list of messages, not just a string prompt
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


#  Function to get completion from messages
#  @param messages: List of messages in the conversation
#  @param model: The model to use for generating the response
#  @param temperature: Controls the randomness of the output
#  @return: The content of the response message
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,  # list of messages
        temperature=temperature,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

Example 1
- Simple chat with AI with first message

In [ ]:

response = get_completion('Hi, my name is Isa')
print(response)
# Maybe Output: "Hello Isa! It's a pleasure to meet you. How can I assist you today?"

Example 2
- If you specify a role or behavior in the system message, the AI bot will adopt that role and maintain it throughout the entire conversation with the user.

In [ ]:
messages = [
    {"role": "system", "content": "You are an assistant that speaks like Shakespeare."},
    {"role": "user", "content": "tell me a joke"},
    {"role": "assistant", "content": "Why did the chicken cross the road"},
    {"role": "user", "content": "I don't know"},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)
# Maybe Output: "To get to the other side, perchance! A jest both old and oft heard, but a merry one nonetheless"

Example 3 
- Just ask for personal information

In [ ]:
messages = [
    {"role": "system", "content": "You are friendly chatbot."},
    {"role": "user", "content": "Yes, can you remind me, what is my name?"},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)
# Maybe Output: "I'm sorry, but I don't have access to your name or any personal information about you. How can I assist you today?"

Example 4 
- Take a list of messages of previous conversion

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [ ]:
import panel as pn  # GUI

pn.extension()

panels = []  # collect display panels

# context is a list of messages that will be sent to the model initially
context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

In [ ]:
# This is the input field for the user to type their message
inp = pn.widgets.TextInput(value="Hi", placeholder="Enter text here…")

# The button to send the message
button_conversation = pn.widgets.Button(name="Chat!")

In [ ]:
# Function to handle the button click event
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ""
    
    # Append the user message to the context
    context.append({"role": "user", "content": f"{prompt}"})
    
    # Get the response from the model
    response = get_completion_from_messages(context)
    
    # Append the assistant's response to the context
    context.append({"role": "assistant", "content": f"{response}"})
    
    # append messages to the panel for the conversation
    panels.append(pn.Row("User:", pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row("Assistant:",pn.pane.Markdown(response, width=600, style={"background-color": "#F6F6F6"})))

    return pn.Column(*panels)

In [ ]:
# Bind the button click event to the collect_messages function
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Create the dashboard layout
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

# Display the dashboard
dashboard

In [ ]:
# After completing the order by chatting with the bot, you can use the following code to display the final order summary:
messages = context.copy()
messages.append(
    {
        "role": "system",
        "content": "create a json summary of the previous food order. Itemize the price for each item\
 The fields should be \
 1) pizza, include size \
 2) list of toppings \
 3) list of drinks, include size   \
 4) list of sides include size  \
 5) total price ",
    },
)
# The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)